# Facility Data Deep Dive

## Violations, Inspections and Enforcements for the Resource Conservation and Recovery Act (RCRA), Clean Water Act (CWA), Clean Air Act (CAA)

This notebook examines data from the EPA's Enforcement and Compliance History Online (ECHO) database (https://echo.epa.gov/). It includes information from EPA's programs covering air (the Clean Air Act, or CAA), water (the Clean Water Act, or CWA) and hazardous and other waste handling (the Resource Recovery and Conservation Act, or RCRA). ECHO data is available here on inspections, violations and enforcements by EPA, state and other agencies. Information made accessible here runs from the present day (the database is refreshed weekly) back to whenever ECHO data begins on a particular program (varying from the 1970s to the 2000s). It uses the following data sets from the ECHO downloadable files:

Hazardous and Other Wastes:
- RCRA_EVALUATIONS = Inspections under RCRA
- RCRA_VIOLATIONS = Violations of RCRA rules
- RCRA_ENFORCEMENTS = Enforcement actions takenby state agencies and the EPA

Cross-Program Compilations:
- ICIS_FEC_EPA_INSPECTIONS = Cross-program, federally-led inspections 
- CASE_FACILITIES = Cross-program enforcements
- ECHO_EXPORTER = A cross-program compilation of all information that is now available on the EPA's public interface for ECHO; mostly limited to the last 3-5 years of data

Air:
- ICIS-AIR_VIOLATION_HISTORY = CAA violations
- ICIS-AIR_FCES_PCES = Both state and federal CAA compliance evaluations
- ICIS-AIR_FORMAL_ACTIONS = CAA formal enforcement actions

Water:
- NPDES_QNCR_HISTORY = CWA Quarterly Non-Compliance History
- NPDES_INSPECTIONS = CWA Inspections
- NPDES_FORMAL_ENFORCEMENT_ACTIONS = CWA Enforcements

Drinking Water:
- SDWA_PUB_WATER_SYSTEMS = Public Water Systems under Safe Drinking Water Act
- SDWA_SITE_VISITS = Safe Drinking Water Site Visits
- SDWA_VIOLATIONS = Safe Drinking Water Violations
- SDWA_SERIOUS_VIOLATORS = Safe Drinking Water Serious Violators
- SDWA_ENFORCEMENTS = Safe Drinking Water Enforcements
- SDWA_RETURN_TO_COMPLIANCE = Safe Drinking Water Return to Compliance


#### For starters, please have available the zip code or of the facility or area in which you area interested.  A zip code must be typed or pasted into  the input widget that is provided after you run the first "cell."

A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.

## How to Run
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---
---

# **Let's begin!**

### Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

### Run this next cell to create the widget to choose whether you want to view facilities by state, county, congressional district or zip code.  Choose the type of region and proceed to the next cell.

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO-modules.git

In [ ]:
# Import libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display

# This is the global function that can run an SQL query against
# the database and return the resulting Pandas DataFrame.
def get_data( sql, index_field=None ):
    url='http://apps.tlt.stonybrook.edu/echoepa/?query='
    data_location=url+urllib.parse.quote(sql)
    # print( data_location )
    ds = pd.read_csv(data_location,encoding='iso-8859-1')
    if ( index_field is not None ):
        ds.set_index( index_field, inplace=True)
    return ds
 
# ECHO_EXPORTER has counties listed both as ALAMEDA and ALAMEDA COUNTY, seemingly
# for every county.  We drop the 'COUNTY' so they only get listed once.
def fix_county_names( in_counties ):
    counties = []
    for county in regions['FAC_COUNTY']:
        if (county.endswith( ' COUNTY' )):
            county = county[:-7]
        counties.append( county.strip() )
    counties = np.unique( counties )
    return counties

region_field = { 
    'County': { "field": 'FAC_COUNTY' },
    'State': { "field": 'FAC_STATE' },
    'Congressional District': { "field": 'FAC_DERIVED_CD113' },
    'Zip Code': { "field": 'FAC_DERIVED_ZIP' }
}

select_region_widget = widgets.Dropdown(
    options=region_field.keys(),
    value='County',
    description='Region of interest:',
    disabled=False
)
display( select_region_widget )

In [ ]:
%load DataSet.py

### Select the state of interest.  If you are interested in a Zip code analysis, this can be skipped.

In [ ]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
dropdown_state=widgets.Dropdown(
    options=states,
    value='NY',
    description='State:',
    disabled=False,
)
output_state = widgets.Output()
my_state = ""

def dropdown_state_eventhandler( change ):
    output_state.clear_output()
    value = change.new
    with output_state:
        display( change.new )
            
dropdown_state.observe( dropdown_state_eventhandler, names='value')
display( dropdown_state )

### If you are looking at a county or congressional district, after selecting a state from the dropdown above, run this cell to pick your region.  
### Run this cell for county, congressional district or zip code regions.  
#### Here is a map of congressional districts: https://www.govtrack.us/congress/members/map

If you are interested in an analysis by state, it is already selected so skip this cell.

In [ ]:
selected_region_type = select_region_widget.value
selected_region_field = region_field[ selected_region_type ]

if ( selected_region_type != 'Zip Code' ):
    my_state = dropdown_state.value
    region_sql = "select distinct("
    region_sql += selected_region_field[ 'field' ] + ")"
    region_sql += " from ECHO_EXPORTER "
    region_sql += " where FAC_STATE = \'" + my_state + "\'"
    try:
        regions = get_data( region_sql )
    except pd.errors.EmptyDataError:
        print("\nThere are no regions of that type in this state.\n")

if ( selected_region_type == 'Zip Code' ):
    region_widget = widgets.IntText(
        value=98225,
        description='Zip Code:',
        disabled=False
    )
elif ( selected_region_type == 'County' ):
    region_widget=widgets.Dropdown(
        options=fix_county_names( regions['FAC_COUNTY'] ),
        description='County:',
        disabled=False
    )
elif ( selected_region_type == 'Congressional District' ):
    region_widget=widgets.Dropdown(
        options=regions['FAC_DERIVED_CD113'].sort_values(),
        description='District:',
        disabled=False
    )
   
if ( selected_region_type != 'State' ):
    display( region_widget )

### Run this cell. It will pull facility information from the data  in the  ECHO_EXPORTER table.

In [ ]:
region_selected = region_widget.value

if ( selected_region_type == 'State' ):
    region_selected = my_state
echo_data_sql = "select * from ECHO_EXPORTER where " + region_field[selected_region_type]['field']
echo_data_sql += " like \'" + str( region_selected) + "%\'"
if ( selected_region_type != 'Zip Code' ):
    echo_data_sql += " and FAC_STATE = \'" + my_state + "\'"
try:
    echo_data = get_data( echo_data_sql, 'REGISTRY_ID' )
    num_facilities = echo_data.shape[0]
    print("\nThere are %s EPA facilities in region %s tracked in the ECHO database." \
          %(num_facilities, str( region_selected )))
except pd.errors.EmptyDataError:
    print("\nThere are no EPA facilities in this region.\n")

## Create an interactive map of all of the facilities in the area that report to EPA
### Once you run this cell, a map should appear. You can zoom in and out, or click on facilities to get their names and the EPA programs that monitor them.

The EPA program acronyms are:
- CAA = Clean Air Act
- CWA = Clean Water Act
- SDWIS = Safe Drinking Water Information System
- RCRA = Resource Conservation and Recovery Act
- TRI = Toxics Release Inventory
- GHG = Greenhouse Gas

The map won't display if there are too many markers, so only the first 400 are shown. 
The facilities shown are selected in order of the ECHO_EXPORTER table's FAC_QTRS_WITH_NC field, which is defined as a "Count of the number of quarters, out of the last twelve quarters, in which the permit or site is considered either in Noncompliance (NC), Significant Noncompliance (SNC) or High Priority violation (HPV) status."

In [ ]:

# Let's show a quick map of your area and the facilities in it
# To-do:  Add some more ECHO_EXPORTER information in the markers.

# Put some information with the marker to show the programs that track the facility.
def marker_text( row ):
    text = ""
    if ( type( row['FAC_NAME'] == str )) :
        try:
            text = row["FAC_NAME"] + ' - '
        except TypeError:
            print( "A facility was found without name. ")
        if ( row['AIR_FLAG'] == 'Y' ):
            text += 'CAA, ' 
        if ( row['NPDES_FLAG'] == 'Y' ):
            text += 'CWA, ' 
        if ( row['SDWIS_FLAG'] == 'Y' ):
            text += 'SDWIS, ' 
        if ( row['RCRA_FLAG'] == 'Y' ):
            text += 'RCRA, ' 
        if ( row['TRI_FLAG'] == 'Y' ):
            text += 'TRI, ' 
        if ( row['GHG_FLAG'] == 'Y' ):
            text += 'GHG, ' 
    return text
    
def mapper(df):
    # Initialize the map
    center = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]]
    m = folium.Map(
        location = center,
    )

    # Add a clickable marker for each facility
    i = 0
    for index, row in df.iterrows():
        # Make sure the FAC_NAME is not NaN, which is interpreted as a number.
        if ( type( row['FAC_NAME'] == str )) :
            folium.Marker(
                location = [row["FAC_LAT"], row["FAC_LONG"]],
                popup = marker_text( row )).add_to(m)
            i += 1
        if ( i > 400 ):    # The map won't display with too many markers.
            break
            
    bounds = m.get_bounds()
    m.fit_bounds(bounds)
    
    # Show the map
    return m

df_to_map = echo_data
if ( len(df_to_map) > 400 ):
    df_to_map = echo_data[echo_data["FAC_QTRS_WITH_NC"] > 0].sort_values(by="FAC_QTRS_WITH_NC", ascending=False)
    
map_of_facilities = mapper(df_to_map)
map_of_facilities

## In this section you may save the region's data to your computer.
This next cell allows you to save the data from ECHO_EXPORTER for your chosen region to your computer.

_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

We need to set up some data structures in this next cell that will be used in the remainder of the report.

## Graph program-specific data for all the facilities in the region.
### First, choose the program (Air, Water, or RCRA) and type of data (Violations, Inspections, Enforcement) you want to explore

Running the code below will show you a dropdown you can use to make a selection. **Inspections** or **evaluations** are usually the first phase in the enforcement pipeline, how many (but not all) violations are discovered.  **Violations** happen when a facilty is found to be out of compliance with an environmental law, often by violating the terms of its permit issued under that law.  **Enforcement actions** happen when the regulating agency decides to initiate a "case" against a firm for its violation of an environmental law, either by considering some administrative penalty such as a fine or by taking the violator to court. 

What's available for you to look at here:

Hazardous and Other Waste:
- RCRA_EVALUATIONS = Inspections under RCRA
- RCRA_VIOLATIONS = Violations of RCRA rules
- RCRA_ENFORCEMENTS = Enforcement actions taken by state agencies and the EPA

Here's where you can find definitions of terms used in the columns of RCRA data: 
https://echo.epa.gov/tools/data-downloads/rcrainfo-download-summary 
- ~~ICIS_FEC_EPA_INSPECTIONS = Cross-program, federally-led inspections~~ (TBD)  
- ~~CASE_FACILITIES = Cross-program enforcements~~ (TBD)

Air:
- ICIS-AIR_VIOLATION_HISTORY = CAA violations
- ICIS-AIR_FCES_PCES = Both state and federal CAA compliance evaluations
- ICIS-AIR_FORMAL_ACTIONS = CAA formal enforcement actions

Here's where you can find definitions of terms used in the columns of Clean Air Act data: 
https://echo.epa.gov/tools/data-downloads/icis-air-download-summary

Water (National Pollutant Discharge Elimination System, or NPDES):
- NPDES_QNCR_HISTORY = CWA Quarterly Non-Compliance History
- NPDES_INSPECTIONS = CWA Inspections
- NPDES_FORMAL_ENFORCEMENT_ACTIONS = CWA Enforcements

Here's where you can find definitions of terms used in the columns of Clean Water Act data: https://echo.epa.gov/tools/data-downloads/icis-npdes-download-summary


Drinking Water (Safe Drinking Water Act, or SDWA):
- SDWA_PUB_WATER_SYSTEMS = Public Water Systems under Safe Drinking Water Act
- SDWA_SITE_VISITS = Safe Drinking Water Site Visits
- SDWA_VIOLATIONS = Safe Drinking Water Violations
- SDWA_SERIOUS_VIOLATORS = Safe Drinking Water Serious Violators
- SDWA_ENFORCEMENTS = Safe Drinking Water Enforcements
- SDWA_RETURN_TO_COMPLIANCE = Safe Drinking Water Return to Compliance

Here's where you can find definitions of terms used in the columns of Clean Water Act data: https://echo.epa.gov/tools/data-downloads/sdwa-download-summary



In [ ]:
# Create a DataSet object for each of the programs we track.  
# Initialize each one with the information it needs to do its query
# of the database.
# Store the DataSet objects in a dictionary with keys being the
# friendly names of the program, which will be used in selection
# widgets.

data_sets = {}
ds = DataSet( name='RCRA Violations', idx_field='ID_NUMBER', 
                table_name='RCRA_VIOLATIONS', echo_type="RCRA",
                date_field='DATE_VIOLATION_DETERMINED', date_format='%m/%d/%Y')
data_sets[ ds.name ] = ds
ds = DataSet( name='RCRA Inspections', idx_field='ID_NUMBER', 
                table_name='RCRA_EVALUATIONS', echo_type="RCRA",
                date_field='EVALUATION_START_DATE', date_format='%m/%d/%Y')
data_sets[ ds.name ] = ds
ds = DataSet( name='RCRA Enforcements',  echo_type="RCRA",
                table_name='RCRA_ENFORCEMENTS', idx_field='ID_NUMBER', 
                date_field='EVALUATION_START_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
# Special sql queries needed for Air Inspections and Air Enforcements
my_sql = "select * from `ICIS_FEC_EPA_INSPECTIONS` where REGISTRY_ID in "
ds = DataSet( name='Air Inspections', echo_type="AIR",
                table_name='ICIS_FEC_EPA_INSPECTIONS', idx_field='REGISTRY_ID', 
                date_field='ACTUAL_END_DATE', date_format='%m/%d/%Y', sql=my_sql )
data_sets[ ds.name ] = ds
my_sql = "select * from `CASE_FACILITIES` CF, `CASE_ENFORCEMENTS` CE " + \
            " where CE.HQ_DIVISION = 'AIR' and CE.CASE_NUMBER = CF.CASE_NUMBER and " + \
            " CF.REGISTRY_ID in "
ds = DataSet( name='Air Enforcements',  echo_type="AIR",
                table_name='CASE_ENFORCEMENTS', idx_field='REGISTRY_ID',
                date_field='FISCAL_YEAR', date_format='%Y', sql=my_sql )
data_sets[ ds.name ] = ds
ds = DataSet( name='Air Violations',  echo_type="AIR",
                table_name='ICIS-AIR_VIOLATION_HISTORY', idx_field='PGM_SYS_ID', 
                date_field='HPV_DAYZERO_DATE', date_format='%m-%d-%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Air Formal Actions', echo_type="AIR",
                table_name='ICIS-AIR_FORMAL_ACTIONS', idx_field='PGM_SYS_ID',
                date_field='SETTLEMENT_ENTERED_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Air Compliance', echo_type="AIR",
                table_name='ICIS-AIR_FCES_PCES', idx_field='PGM_SYS_ID',
                date_field='ACTUAL_END_DATE', date_format='%m-%d-%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Water Quarterly Violations', echo_type="NPDES",
                table_name='NPDES_QNCR_HISTORY', idx_field='NPDES_ID',
                date_field='YEARQTR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Clean Water Inspections', echo_type="NPDES",
                table_name='NPDES_INSPECTIONS', idx_field='NPDES_ID',
                date_field='ACTUAL_END_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='Clean Water Enforcements', echo_type="NPDES",
                table_name='NPDES_FORMAL_ENFORCEMENT_ACTIONS', idx_field='NPDES_ID',
                date_field='SETTLEMENT_ENTERED_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Site Visits', echo_type="SDWA",
                table_name='SDWA_SITE_VISITS', idx_field='PWSID',
                date_field='SITE_VISIT_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Enforcements', echo_type="SDWA",
                table_name='SDWA_ENFORCEMENTS', idx_field='PWSID',
                date_field='ENFORCEMENT_DATE', date_format='%m/%d/%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Public Water Systems', echo_type="SDWA",
                table_name='SDWA_PUB_WATER_SYSTEMS', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Violations', echo_type="SDWA",
                table_name='SDWA_VIOLATIONS', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Serious Violators', echo_type="SDWA",
                table_name='SDWA_SERIOUS_VIOLATORS', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )
data_sets[ ds.name ] = ds
ds = DataSet( name='SDWA Return to Compliance', echo_type="SDWA",
                table_name='SDWA_RETURN_TO_COMPLIANCE', idx_field='PWSID',
                date_field='FISCAL_YEAR', date_format='%Y' )

# Only list the data set if it has the correct flag set.
data_set_choices = []
for k, v in data_sets.items():
    if ( v.has_echo_flag( echo_data ) ):
        data_set_choices.append( k )

# data_set_choices = list( data_sets.keys() )

data_set_widget=widgets.Dropdown(
    options=list(data_set_choices),
    description='Data sets:',
    disabled=False,
) 
display(data_set_widget)

### Once you've chosen the program and data type you want to focus on...
....run the code below, which will access our database and retrieve the information

In [ ]:
program = data_sets[ data_set_widget.value ]
ids_string = program.echo_type + '_IDS'
ids = list()
for index, value in echo_data[ ids_string ].items():
    try:
        for npdes_id in value.split():
            ids.append( npdes_id )
    except ( KeyError, AttributeError ) as e:
        pass
ids

In [ ]:
'LAG670191' in ids


In [ ]:
program = data_sets[ data_set_widget.value ]

ids_string = program.echo_type + '_IDS'
ids = list()
for index, value in echo_data[ ids_string ].items():
    try:
        for npdes_id in value.split():
            ids.append( npdes_id )
    except ( KeyError, AttributeError ) as e:
        pass

program_data = None
# We need to provide a custom sql query and argument for these programs.
if ( program.name == "Air Inspections" or program.name == "Air Enforcements" ):
    # The REGISTRY_ID field is the index of the echo_data
    # Build a string
    registry_ids = echo_data[echo_data['AIR_FLAG'] == 'Y'].index.to_list()
    program_data = program.get_data( ee_ids=registry_ids )
else:
    program_data = program.get_data( ee_ids=ids )

program_data

## In this section you may save the program data for your region to your computer.
This next cell allows you to save the data from the particular program for your chosen region to your computer.

_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

In [ ]:
filename = program.name + '-' + str( region_selected )
fullpath = filename+'.csv'
program_data.to_csv( fullpath ) 

print( "Wrote "+filename+" to the Google Colab 'Files' menu as %s" %(fullpath))

### Let's make a chart out of this!

In [ ]:
chart_title = program.name + ' - ' + str( region_selected )

# Handle NPDES_QNCR_HISTORY because there are multiple counts we need to sum
if (program.name == "Water Quarterly Violations"): 
    year = program_data["YEARQTR"].astype("str").str[0:4:1]
    program_data["YEARQTR"] = year
    d = program_data.groupby(pd.to_datetime(program_data['YEARQTR'], format="%Y").dt.to_period("Y")).sum()
    d.index = d.index.strftime('%Y')
    d = d[ d.index > '2000' ]
    
    ax = d.plot(kind='bar', title = chart_title, figsize=(20, 10), fontsize=16)
    ax
# These data sets use a FISCAL_YEAR field
elif (program.name == "SDWA Public Water Systems" or program.name == "SDWA Violations" or
     program.name == "SDWA Serious Violators" or program.name == "SDWA Return to Compliance"):
    year = program_data["FISCAL_YEAR"].astype("str")
    program_data["FISCAL_YEAR"] = year
    d = program_data.groupby(pd.to_datetime(program_data['FISCAL_YEAR'], format="%Y").dt.to_period("Y"))[['PWS_NAME']].count()
    d.index = d.index.strftime('%Y')
    d = d[ d.index > '2000' ]
    
    ax = d.plot(kind='bar', title = chart_title, figsize=(20, 10), fontsize=16)
    ax        
# All other columns
else:
    try:
        d = program_data.groupby(pd.to_datetime(program_data[program.date_field], format=program.date_format))[[program.date_field]].count()
        d = d.resample("Y").count()
        d.index = d.index.strftime('%Y')
        d = d[ d.index > '2000' ]
        
        ax = d.plot(kind='bar', title = chart_title, figsize=(20, 10), legend=False, fontsize=16)
        ax
        
    except AttributeError:
        print("There's no data to chart for " + program.name + " !")

### Let's map just the facilities in this selected region monitored under this program.

In [ ]:
if ( program.echo_type == 'SDWA' ):
    flag = 'SDWIS_FLAG'
else:
    flag = program.echo_type + '_FLAG'
df_to_map = echo_data[ echo_data[ flag ] == 'Y' ]
map_of_facilities = mapper(df_to_map)
map_of_facilities

### If you'd like to look at another dimension of ECHO data for this region...
....return to the dropdown menu above and choose a different program/data type, run the database access code, and then chart and map it!

---

## Choose a specific facility
### In the next few blocks of code, you can focus in on just one of these facilities in the region.
Run the below cell, then choose a facility from the dropdown that appears.

In [ ]:
cd_array = echo_data["FAC_NAME"].dropna().unique()
cd_array.sort()

fac_widget=widgets.Dropdown(
    options=cd_array,
    description='Facility Name:',
    disabled=False,
)
display(fac_widget)

### The next cell filters the program-specific IDs to get just records for the selected facility.

### Show the information from one of the data sets for this facility
Running this cell will create a dropdown list of the program data sets available specifically for this faciliity. Pick one and then run the second cell to view its contents.

In [ ]:
my_fac_name

In [ ]:
my_fac_name = fac_widget.value
my_fac = echo_data[echo_data["FAC_NAME"] == my_fac_name]

data_set_choices = []
for k, v in data_sets.items():
    if ( v.has_echo_flag( my_fac ) ):
        data_set_choices.append( k )

data_set_widget2=widgets.Dropdown(
    options=data_set_choices,
    description='Data sets:',
    disabled=False,
)
display(data_set_widget2)


### Let's chart it!

In [ ]:
fac_program = data_sets[ data_set_widget2.value ]

ids_string = fac_program.echo_type + '_IDS'
ids = my_fac.loc[my_fac[ids_string].str.len() >0]    # just give all _IDs and let sql deal with it?
ids = ids.loc[:,ids_string].unique()

# We need to provide a custom sql query and argument for these programs.
if ( fac_program.name == "Air Inspections" or program.name == "Air Enforcements" ):
    # The REGISTRY_ID field is the index of the echo_data
    # Build a string
    registry_ids = my_fac[my_fac['AIR_FLAG'] == 'Y'].index.to_list()
    # breakpoint()
    this_data = fac_program.get_data( ee_ids=registry_ids )
else:
    this_data = fac_program.get_data( ee_ids=ids )
if ( this_data is None ):
    print( "There is no {} data for {} ".format( fac_program.name, my_fac_name ))
else:
    print( this_data )

In [ ]:
# Handle NPDES_QNCR_HISTORY because there are multiple counts we need to sum
fac_program_data = this_data
chart_title = fac_program.name + ' - ' + my_fac_name

if (fac_program.name == "Water Quarterly Violations"): 
    year=fac_program_data["YEARQTR"].astype("str").str[0:4:1]
    fac_program_data["YEARQTR"]=year
    fac_program_data = fac_program_data.groupby(pd.to_datetime(this_data['YEARQTR'], format="%Y").dt.to_period("Y")).sum()
    fac_program_data.index = fac_program_data.index.strftime('%Y')
    fac_program_data = fac_program_data[ fac_program_data.index > '2000' ]
    
    ax = fac_program_data.plot(kind='bar', title = chart_title, figsize=(20, 10), fontsize=16)
    ax
    
# These data sets use a FISCAL_YEAR field
elif (fac_program.name=="SDWA Water Systems" or fac_program.name=="SDWA Violations" or
     fac_program.name=="SDWA Serious Violators" or fac_program.name=="SDWA Return to Compliance"):
    year = fac_program_data["FISCAL_YEAR"].astype("str")
    fac_program_data["FISCAL_YEAR"] = year
    fac_program_data = fac_program_data.groupby(pd.to_datetime(fac_program_data['FISCAL_YEAR'], 
                                                       format="%Y").dt.to_period("Y"))[['PWS_NAME']].count()
    fac_program_data.index = fac_program_data.index.strftime('%Y')
    fac_program_data = fac_program_data[ fac_program_data.index > '2000' ]
    
    ax = fac_program_data.plot(kind='bar', title = chart_title, figsize=(20, 10), fontsize=16)
    ax        

# All other data sets
else: 
    try:
        fac_program_data = fac_program_data.groupby(pd.to_datetime(fac_program_data[fac_program.date_field], 
                                                           format=fac_program.date_format))[[fac_program.date_field]].count()
        fac_program_data = fac_program_data.resample("Y").count()
        fac_program_data.index = fac_program_data.index.strftime('%Y')
        fac_program_data = fac_program_data[ fac_program_data.index > '2000' ]
        
        ax = fac_program_data.plot(kind='bar', title = chart_title, 
                               figsize=(20, 10), legend=False, fontsize=16)
        ax
        
    except AttributeError:
        print("There's no {} data to chart for {}!".format( fac_program.name, my_fac_name ))

## This section saves this facility's data for this program to your computer.
The next cell exports this data to your computer.  You can return to the dropdown to export additional data files.

_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

In [ ]:
filename = fac_program.name + '-' + my_fac_name
fullpath = filename+'.csv'
this_data.to_csv( fullpath ) 

print( "Wrote "+filename+" to the Google Colab 'Files' menu as %s" %(fullpath))

### Accessing your files
Click on the 'Files' tab in the menu on the left-hand side of the notebook (it looks like a folder). You may have to hit 'Refresh' if you don't see your file. Then, you can click on the ... next to your file and choose "Download". The CSV spreadsheet will download to wherever your browser usually saves files (e.g. Downloads folder)

### If you'd like to look at another dimension of ECHO data for this facility...
....return to the dropdown menu above and choose a different program/data type, then chart it!